In [ ]:
import os
import sys
import re
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import pickle
import seaborn as sns
import json
import copy
import logging
from itertools import product
from IPython.display import Markdown, display
logging.basicConfig(level=logging.INFO)
logging.getLogger("matplotlib").setLevel(logging.INFO)
logger = logging.getLogger(__name__)

sys.path.append("..")
import pandas as pd
from util.distance import dist3
from util.scenario import Scenario

box_color = '#86afb8'

In [ ]:
def get_nr_charge_cycles(loaded):
    nr_drones = loaded['scenario']['nr_drones']

    res = []
    for d in range(nr_drones):
        charge_cycles = 0
        currently_charging = False
        for ev in loaded['events'][d]:
            if ev['type'] == 'charged':
                currently_charging = True            
            elif currently_charging: 
                # stop charging            
                charge_cycles += 1
                currently_charging = False
        res.append(charge_cycles)
    return res
    

In [ ]:
def load_data_from_json(basedir):
    data = []
    for subdir in os.listdir(basedir):
        if os.path.isdir(os.path.join(basedir, subdir)):
            result_fpath = os.path.join(basedir, subdir, "result.json")
            if not os.path.exists(result_fpath):
                logger.info(f"skipping '{subdir}' due to missing 'result.json' file")
                continue                
            
            with open(result_fpath, 'r') as f:
                try:
                    loaded = json.load(f)
                except Exception as e:
                    logger.error(f"failed to load JSON for '{subdir}': {e}")
                    continue
                
                datapoint = {}

                # parse parameters
                params = loaded['params']
                
                cols = ['v', 'r_charge', 'r_deplete', 'B_max', 'B_min', 'epsilon', 'schedule_delta', 'W', 'sigma', 'time_limit', 'int_feas_tol', 'rescheduling_frequency']
                for col in cols:
                    val = params[col]
                    if type(val) == list:
                        val = val[0]
                    datapoint[col] = val              
                    
                # parse relevant results
                datapoint['scheduler'] = loaded['scheduler']
                datapoint['dirname'] = subdir
                datapoint['execution_time'] = loaded['execution_time']
                datapoint['nr_drones'] = loaded['scenario']['nr_drones']
                datapoint['solve_times'] = [x['t_solve'] for x in loaded['solve_times']]
                datapoint['solve_timestamps'] = [x['timestamp'] for x in loaded['solve_times']]
                datapoint['solve_optimality'] = [x['optimal'] for x in loaded['solve_times']]
                datapoint['moving_times'] = [x['moving'] for x in loaded['time_spent'].values()]
                datapoint['waiting_times'] = [x['waiting'] for x in loaded['time_spent'].values()]
                datapoint['charging_times'] = [x['charging'] for x in loaded['time_spent'].values()]
                datapoint['moving_times_minimum'] = [x['moving_minimum'] for x in loaded['time_spent'].values()]
                datapoint['total_occupancy'] = [sum(x['t_end'] - x['t_start'] for x in l) for l in loaded['occupancy'].values()]
                datapoint['nr_charge_cycles'] = get_nr_charge_cycles(loaded)
                
                data.append(datapoint)
    res = pd.DataFrame(data)
    
    # preprocessing
    # basic statistical functions
    cols = ['moving_times', 'waiting_times', 'charging_times', 'solve_times', 'nr_charge_cycles']
    for col in cols:
        new_col = f"{col}_cum"
        res[new_col] = res[col].apply(sum)
        
        new_col = f"{col}_mean"
        res[new_col] = res[col].apply(np.mean)
        
        new_col = f"{col}_min"
        res[new_col] = res[col].apply(np.min)
        
        new_col = f"{col}_max"
        res[new_col] = res[col].apply(np.max)

    # counts
    cols = ['solve_times']
    for col in cols:
        new_col = f"{col}_count"
        res[new_col] = res[col].apply(len)
        
    # percent spent on different actions
    cols = ['moving_times', 'waiting_times', 'charging_times']
    total = res[cols].apply(lambda x: sum([np.array(v) for v in x]), axis=1)
    for col in cols:
        new_col = f"{col}_perc"
        res[new_col] = res[col].apply(np.array) / total * 100
        
        new_mean_col = f"{col}_perc_mean"
        res[new_mean_col] = res[new_col].apply(np.mean)
        
    res['perc_occupancy'] = res.apply(lambda x: np.array(x['total_occupancy']) / x['execution_time'] * 100, axis=1)    
    res['solve_optimality_perc'] = res.solve_optimality.apply(lambda x: np.sum(x) / len(x) * 100)
    
    # strings
    res['scheduler'] = res.scheduler.apply(lambda x: x[:-9])
    
    # column renaming
    res = res.rename(columns={'rescheduling_frequency': 'pi'})
    
    res['h'] = (res.W - 1) * res.sigma
    res['perc_max_horizon_executed'] = res['pi'] / res['h'] * 100
    res['solve_times_optimal_mean'] = res.solve_times.apply(lambda x: np.mean([e for e in x if e < 10]))
    
    # TOOD: extract nr charging cycles
    
    return res

In [ ]:
basedir = "../out/villalvernia/grid_search"
df_planning_horizon = load_data_from_json(basedir)
df_planning_horizon_milp = df_planning_horizon[lambda x: x.scheduler == 'milp']
df_planning_horizon_naive = df_planning_horizon[lambda x: x.scheduler == 'naive']
naive_execution_time = df_planning_horizon_naive.iloc[0].execution_time
df_planning_horizon_milp['perc_execution_time_naive'] = df_planning_horizon_milp.execution_time / naive_execution_time * 100
df_planning_horizon_milp['max_anchor_count'] = df_planning_horizon_milp.apply(lambda x: np.floor(x.W / x.sigma), axis=1).astype(int)
logger.info(f"Number of data points: {df_planning_horizon.shape[0]}")

In [ ]:
plot_cols = ['W', 'pi', 'sigma']

pis = sorted(df_planning_horizon_milp.pi.unique())
ws = sorted(df_planning_horizon_milp.W.unique())
sigmas = sorted(df_planning_horizon_milp.sigma.unique())

ys = ['perc_execution_time_naive', 'execution_time', 'solve_optimality_perc', 'solve_times_cum']
labels = ['Relative execution\ntime (% vs naive)', 'Execution\ntime (s)', 'Optimal\nsolves (%)', 'Total solve time\n(s, log)']

pi_count = len(pis)
w_count = len(ws)
sigma_count = len(sigmas)

figsize = w_count * sigma_count * 1.2, 3

dataframes = []

for i, y in enumerate(ys):
    fig, axes = plt.subplots(nrows=1, ncols=pi_count, figsize=figsize, sharey=True)
    for j, pi in enumerate(pis):
        ax = axes[j]
        
        # prepare plotting dataframe
        df_plot = df_planning_horizon_milp[lambda x: x.pi == pi][plot_cols + [y]]        
        new_rows = set(product(ws, sigmas)) - set(tuple(a) for a in df_plot[['W', 'sigma']].values)
        df_new_rows = pd.DataFrame(data=new_rows, columns=['W', 'sigma'])
        df_plot_new_rows = pd.concat([df_plot, df_new_rows])
        
        dataframes.append(df_plot_new_rows)
        sns.barplot(x='W', y=y, hue='sigma', data=df_plot_new_rows, ax=ax, edgecolor='black', linewidth=0.5)
        ax.grid(axis='y')
        if j == 0:
            ax.set_ylabel(labels[i])
        else:
            ax.set_ylabel("")
        # fix legend
        box = ax.get_position()
        ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])    
        ax.legend(title="$\sigma$", loc='center left', bbox_to_anchor=(1, 0.5))
        
        # disable axis (top and right)
        ax.spines.right.set_visible(False)
        ax.spines.top.set_visible(False)
        ax.set_title(f"$\pi = {pi}$")

    fname = f"../out/figures/gridsearch/barplots_{y}.pdf"
    plt.savefig(fname, bbox_inches="tight")
    plt.show()